#### imports

In [ ]:
import numpy as np 
import pandas as pd
import re
import string
from gensim.models import Word2Vec

import tensorflow.keras as tfk

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras import layers

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
# we are only submitting the notebook, so you will likley need your own kaggle API Token
# file (json) to be able to run these commands. read more about it here:
# https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! kaggle datasets download -d oliveirasp6/poems-in-portuguese
! unzip poems-in-portuguese.zip
! mv portuguese-poems.csv dataset-portuguese.csv

 91% 5.00M/5.50M [00:00<00:00, 20.2MB/s]
100% 5.50M/5.50M [00:00<00:00, 22.0MB/s]
Archive:  poems-in-portuguese.zip
  inflating: portuguese-poems.csv    


In [ ]:
! kaggle datasets download -d andreamorgar/spanish-poetry-dataset
! unzip spanish-poetry-dataset.zip
! mv poems.csv dataset-spanish.csv

  0% 0.00/2.12M [00:00<?, ?B/s]
100% 2.12M/2.12M [00:00<00:00, 147MB/s]
Archive:  spanish-poetry-dataset.zip
  inflating: poems.csv               


In [ ]:
! kaggle datasets download -d watrgoat/lots-of-poems
! unzip lots-of-poems.zip
! mv mostly_english_poems.csv dataset-english.csv

 81% 49.0M/60.4M [00:01<00:00, 38.8MB/s]
100% 60.4M/60.4M [00:01<00:00, 41.8MB/s]
Archive:  lots-of-poems.zip
  inflating: mostly_english_poems.csv  
  inflating: mostly_english_poems.pickle  
  inflating: ready_poems.csv         
  inflating: ready_poems.pickle      


# Data Sets + Preprocessing 

In [ ]:
lang_class = {'portuguese': 0, 'spanish': 1, 'english': 2, 
              0: 'portuguese', 1: 'spanish', 2: 'english'}
lang_class_other  = {'portuguese': 2, 'spanish': 1, 'english': 0, 
              2: 'portuguese', 1: 'spanish', 0: 'english'}

In [ ]:
EMBEDDINGS_SIZE = 100
SCALE_LINES = 151000
NGRAM = 3 

In [ ]:
def seperate_lines(content):
  """
  content: a list of chunks of text in the format [['text\nmoret text] , ...]
  
  return: a list of list of lines of text from the content 
            seperated by new line tokens in the format
            ['text', 'moret text' , ...]
  """
  strip_lines = []
  for poem in content:
      if len(poem.strip()) == 0:
          continue
      sent_data = [x for x in re.split(r'\n|\r|/|//|»|\r\n', poem)  if x] 
      #these were some of the new line tokens I identified
      for i in range(len(sent_data)):
          # remove any extraneous whitespace
          sent_data[i] = sent_data[i].strip()
      t = tuple(sent_data)
      strip_lines += list(t)
  return strip_lines


def tokenize_lines(lines):
  """
  lines: list of lines in a text. 
         pre split on end line tokens  in the format (not cleaned
         for excess punctuation)
         [['text', 'moret text'] , ...]

  return tokenized_lines: an list of sentences represented as a list
              of tokens with start and end tokens <s> and </s>
              in the format [[<s>, text, </s], [<s>, moret, text, </s>], ...]
        cleaned_lines : a list of lines of text with the same format as input
               but cleaned of extra punctuation
  """
  tokenized_lines = []
  cleaned_lines = []
  
  for line in lines:
      line_words = line.split()
      line_words_clean = []
      for lw in line_words: # ty geeks for geeks
        line_words_clean.append(lw.translate(str.maketrans('', '', string.punctuation+"©«®¼½÷“”…™←→−≠¿¡" + string.digits)))
      start_tok = "<s>"
      end_tok = "</s>"
      sentence_clean = " ".join(line_words_clean)
      line_words_clean.insert(0, start_tok)
      line_words_clean.append(end_tok)
      tokenized_lines.append(line_words_clean)
      cleaned_lines.append(sentence_clean)

  return tokenized_lines, cleaned_lines


## Portuguese

**TODO some explanation**

https://www.kaggle.com/datasets/oliveirasp6/poems-in-portuguese 
15,543 texts by the most renowned Brazilian authors gathered from the website https://www.escritas.org/pt/poemas.

In [ ]:
dataset_portuguese = pd.read_csv('dataset-portuguese.csv')
dataset_portuguese.head()

,Author,Title,Content,Views
0,Cecília Meireles,Retrato,"Eu não tinha este rosto de hoje,\r\nAssim calm...",1018431
1,Fernando Pessoa,"Para ser grande, sê inteiro: nada","Para ser grande, sê inteiro: nada\r\nTeu exage...",1979413
2,Marina Colasanti,"Eu sei, mas não devia",Eu sei que a gente se acostuma. Mas não devia....,301509
3,Carlos Drummond de Andrade,Quadrilha,João amava Teresa que amava Raimundo\r\nque am...,1421206
4,Eugénio de Andrade,É urgente o amor,É urgente o amor.\r\nÉ urgente um barco no mar...,621197


In [ ]:
#drop NA content, reindex if any was dropped
print(dataset_portuguese.count())
dataset_portuguese.dropna(subset=['Content'],inplace=True)
dataset_portuguese.reset_index(drop=True,inplace=True)
print(dataset_portuguese.count())
#lol there werent any just checking

Author     15543
Title      15543
Content    15541
Views      15543
dtype: int64
Author     15541
Title      15541
Content    15541
Views      15541
dtype: int64


In [ ]:
poems_portuguese = dataset_portuguese[['Content']]
poems_portuguese.head()

,Content
0,"Eu não tinha este rosto de hoje,\r\nAssim calm..."
1,"Para ser grande, sê inteiro: nada\r\nTeu exage..."
2,Eu sei que a gente se acostuma. Mas não devia....
3,João amava Teresa que amava Raimundo\r\nque am...
4,É urgente o amor.\r\nÉ urgente um barco no mar...


In [ ]:
poems_portuguese.insert(1, "Language", lang_class['portuguese'])
poems_portuguese.head()

,Content,Language
0,"Eu não tinha este rosto de hoje,\r\nAssim calm...",0
1,"Para ser grande, sê inteiro: nada\r\nTeu exage...",0
2,Eu sei que a gente se acostuma. Mas não devia....,0
3,João amava Teresa que amava Raimundo\r\nque am...,0
4,É urgente o amor.\r\nÉ urgente um barco no mar...,0


In [ ]:
content_portuguese = poems_portuguese['Content'].array
print(len(content_portuguese))
# content_portuguese

15541


In [ ]:
# things to note: roman numerals, uh theres random german? 
# and other translations mixed in? the german was seperated by // or ....... (which seems to be \n charachter)
# from what I saw
lines_portuguese = seperate_lines(content_portuguese)
# print(lines_portuguese)
print(len(lines_portuguese))


317296


In [ ]:
tokenized_portuguese, cleaned_portuguese = tokenize_lines(lines_portuguese)
# tokenized_portuguese
print(len(tokenized_portuguese))

317296


In [ ]:
word2vec_portuguese = Word2Vec(sentences=tokenized_portuguese, vector_size=EMBEDDINGS_SIZE, window=5, min_count=1, workers=4, sg=1)
word2vec_portuguese.save("word2vec_portuguese.model")
print(word2vec_portuguese.wv.vectors.shape)

(160141, 100)


##Spanish

**TODO some explanation**

https://www.kaggle.com/datasets/andreamorgar/spanish-poetry-dataset Spanish poems gathered from www.poemas-del-alma.com each with the name of the poet, poem, and poem title. 

In [ ]:
dataset_spanish = pd.read_csv('dataset-spanish.csv')
dataset_spanish.head()

,author,content,title
0,Leopoldo Lugones,\n\nEn el parque confuso\nQue con lánguidas br...,LA MUERTE DE LA LUNA
1,Marilina Rébora,"\n\nPorque si tú no velas, vendré como ladrón;...",PORQUE SI TÚ NO VELAS
2,Antonio Colinas,"\n\nPequeña de mis sueños, por tu piel las pal...",POEMA DE LA BELLEZA CAUTIVA QUE PERDÍ
3,José María Hinojosa,\n\nLos dedos de la nieve\nrepiquetearon\nen e...,SENCILLEZ
4,Rubén Izaguirre Fiallos,"Naciste en Armenia,\npero te fuiste a vivir al...",Breve Carta a Consuelo Suncín


In [ ]:
#drop NA content, reindex if any was dropped
print(dataset_spanish.count())
dataset_spanish.dropna(subset=['content'],inplace=True)
dataset_spanish.reset_index(drop=True,inplace=True)
print(dataset_spanish.count())

author     5131
content    5131
title      5131
dtype: int64
author     5131
content    5131
title      5131
dtype: int64


In [ ]:
poems_spanish = dataset_spanish[['content']]
poems_spanish = poems_spanish.rename(columns={'content': 'Content'}) 
poems_spanish.head()

,Content
0,\n\nEn el parque confuso\nQue con lánguidas br...
1,"\n\nPorque si tú no velas, vendré como ladrón;..."
2,"\n\nPequeña de mis sueños, por tu piel las pal..."
3,\n\nLos dedos de la nieve\nrepiquetearon\nen e...
4,"Naciste en Armenia,\npero te fuiste a vivir al..."


In [ ]:
poems_spanish.insert(1, "Language", lang_class['spanish'])
poems_spanish.head()

,Content,Language
0,\n\nEn el parque confuso\nQue con lánguidas br...,1
1,"\n\nPorque si tú no velas, vendré como ladrón;...",1
2,"\n\nPequeña de mis sueños, por tu piel las pal...",1
3,\n\nLos dedos de la nieve\nrepiquetearon\nen e...,1
4,"Naciste en Armenia,\npero te fuiste a vivir al...",1


In [ ]:
content_spanish = poems_spanish['Content'].array
print(len(content_spanish))
# content_spanish

5131


In [ ]:
# things to note: only saw spanish and \n seperated lines 
# am also actually seeing some english 
# Nnoticing some \x97 and » dont know what it is? line separaator?
lines_spanish = seperate_lines(content_spanish)
# lines_spanish
print(len(lines_spanish))

151125


In [ ]:
tokenized_spanish, cleaned_spanish = tokenize_lines(lines_spanish)
# tokenized_spanish
print(len(tokenized_spanish))
# TODO : maybe we should scale the data down here a bit
# note: we strip punctuation but punctuation could be indicative ¿ 

151125


In [ ]:
word2vec_spanish = Word2Vec(sentences=tokenized_spanish, vector_size=EMBEDDINGS_SIZE, window=5, min_count=1, workers=4, sg=1)
word2vec_spanish.save("word2vec_spanish.model")
print(word2vec_spanish.wv.vectors.shape)

(63119, 100)


## English

**TODO some explanation**

https://www.kaggle.com/datasets/watrgoat/lots-of-poems the poems are from the poetry foundation website. 

**Theres two options for data for us to use. theres mostly_english_poems.csv and ready_poems.csv. ready CSV has some <NEWLINE> characters already preprocessed that we could split on. we need to make a choice there**

In [ ]:
dataset_english = pd.read_csv('dataset-english.csv')
dataset_english.head()

,Unnamed: 0,title,author,content,tags
0,0,Do not go gentle into that good night,Dylan Thomas,"Do not go gentle into that good night,\nOld ag...",[]
1,1,How Do I Love Thee? (Sonnet 43),Elizabeth Barrett Browning,How do I love thee? Let me count the ways.\nI ...,[]
2,2,Shall I compare thee to a summer’s day? (Sonne...,William Shakespeare,Shall I compare thee to a summer’s day?\nThou ...,[]
3,3,If—,Rudyard Kipling,If you can keep your head when all about you\n...,[]
4,4,Nothing Gold Can Stay,Robert Frost,"Nature’s first green is gold,\nHer hardest hue...",[]


In [ ]:
#drop NA content, reindex if any was dropped
print(dataset_english.count())
dataset_english.dropna(subset=['content'],inplace=True)
dataset_english.reset_index(drop=True,inplace=True)
print(dataset_english.count())

Unnamed: 0    26817
title         26817
author        26796
content       26817
tags          26817
dtype: int64
Unnamed: 0    26817
title         26817
author        26796
content       26817
tags          26817
dtype: int64


In [ ]:
poems_english = dataset_english[['content']]
poems_english = poems_english.rename(columns={'content': 'Content'}) 
poems_english.head()

,Content
0,"Do not go gentle into that good night,\nOld ag..."
1,How do I love thee? Let me count the ways.\nI ...
2,Shall I compare thee to a summer’s day?\nThou ...
3,If you can keep your head when all about you\n...
4,"Nature’s first green is gold,\nHer hardest hue..."


In [ ]:
poems_english.insert(1, "Language", lang_class['english'])
poems_english.head()

,Content,Language
0,"Do not go gentle into that good night,\nOld ag...",2
1,How do I love thee? Let me count the ways.\nI ...,2
2,Shall I compare thee to a summer’s day?\nThou ...,2
3,If you can keep your head when all about you\n...,2
4,"Nature’s first green is gold,\nHer hardest hue...",2


In [ ]:
content_english = poems_english['Content'].array
# print(len(content_english))
# content_english

In [ ]:
# things to note:
lines_english = seperate_lines(content_english)
# lines_english
print(len(lines_english))

901433


In [ ]:
tokenized_english, cleaned_english = tokenize_lines(lines_english)
# tokenized_english
print(len(tokenized_english))
# TODO : maybe we should scale the data down here a bit

901433


In [ ]:
word2vec_english = Word2Vec(sentences=tokenized_english, vector_size=EMBEDDINGS_SIZE, window=5, min_count=1, workers=4, sg=1)
word2vec_english.save("word2vec_english.model")
print(word2vec_english.wv.vectors.shape)

(215313, 100)


## Some Data Analysis

In [ ]:
# TODO do the word to vec comparison for spanish and english etc.

# Naive Bayes

## training

In [ ]:
train_NB_portuguese = cleaned_portuguese[0:round(SCALE_LINES/2)]
train_NB_spanish = cleaned_spanish[0:round(SCALE_LINES/2)]
train_NB_english = cleaned_english[0:round(SCALE_LINES/2)]
train_NB_corpus = train_NB_portuguese + train_NB_spanish + train_NB_english
print(len(train_NB_corpus))

train_NB_gold = [lang_class['portuguese'] for i in range(round(SCALE_LINES/2))]
train_NB_gold += [lang_class['spanish'] for i in range(round(SCALE_LINES/2))]
train_NB_gold += [lang_class['english'] for i in range(round(SCALE_LINES/2))]
print(len(train_NB_gold))

test_NB_portuguese = cleaned_portuguese[-(round(SCALE_LINES/2)):]
test_NB_spanish = cleaned_spanish[-(round(SCALE_LINES/2)):]
test_NB_english = cleaned_english[-(round(SCALE_LINES/2)):]
test_NB_corpus = test_NB_portuguese + test_NB_spanish + test_NB_english
print(len(test_NB_corpus))
test_NB_gold = train_NB_gold
print(len(test_NB_gold))

In [ ]:
NB_vectorizer = TfidfVectorizer()
# TODO i wonder how adding stop words would affect the model?
train_NB_X = NB_vectorizer.fit_transform(train_NB_corpus)
print(train_NB_X.shape)

In [ ]:
# print(NB_vectorizer.get_feature_names_out())
# print(NB_vectorizer.get_feature_names_out()[6000:6900])
# i can see the portuguese c and also different spanish accents in the words
# why are there random unicode characters and chinese characters?
# TODO (resolved) unsure if need to chaneg encoding to ISO-8859-1 or if utf 8 is fine
# both look fine in terms of output here

In [ ]:
NB_model = MultinomialNB()
#train
NB_model.fit(train_NB_X, train_NB_gold)

In [ ]:
#predict on train data
train_NB_pred = NB_model.predict(train_NB_X)

In [ ]:
mat = confusion_matrix(train_NB_gold, train_NB_pred)
sns.heatmap(mat.T, square = True, annot=True, fmt = "d", xticklabels=[ lang_class[0], lang_class[1], lang_class[2]],yticklabels=[lang_class[0], lang_class[1], lang_class[2]])
plt.xlabel("true labels")
plt.ylabel("predicted label")
plt.title("Training Poems Performance")
plt.show()
print("The accuracy is {}".format(accuracy_score(train_NB_gold, train_NB_pred)))

In [ ]:
def test_model_and_graph(model, classes, sentences, gold, graph_title):
  """
  model: the NB model to test on 
  lang_class: a dictonary of the string representations of the classes to their numerical class representation (and vice versa)
  sentences: a list of strings to classify 
  gold: a list of numercal class value representatiosn that correspond to the sentences
  graph_title : string of what to name the graph
  """
  X = NB_vectorizer.transform(sentences)
  pred = model.predict(X)
  print("The accuracy is {}".format(accuracy_score(gold, pred)))
  mat = confusion_matrix(gold, pred)
  sns.heatmap(mat.T, square = True, annot=True, fmt = "d", xticklabels=[classes[0], classes[1], classes[2]],yticklabels=[classes[0], classes[1], classes[2]])
  plt.title(graph_title)
  plt.xlabel("true labels")
  plt.ylabel("predicted label")
  plt.show()

In [ ]:
test_model_and_graph(NB_model, lang_class, test_NB_corpus, test_NB_gold, "Test Poems Performance")

## manual testing

In [ ]:
# TODO some small little tests

def NBmodel_smol_tests(model, classes, sentences):
  X = NB_vectorizer.transform(sentences)
  pred = model.predict(X)
  [print(sentences[i], "is", classes[pred[i]]) for i in range(len(sentences))]

NBmodel_smol_tests(NB_model, lang_class, ["Nicole", "Sierra", "Buckley", "Rodriguez"])
NBmodel_smol_tests(NB_model, lang_class, ["hello", "hola", "olá"])
NBmodel_smol_tests(NB_model, lang_class, ["good morning", "buenos días", "bom dia"])

## stop words testing

I was just curious how naive bayes would allocate words that are very common to the respective languages 

In [ ]:
! kaggle datasets download -d heeraldedhia/stop-words-in-28-languages
! unzip stop-words-in-28-languages.zip 

In [ ]:
f = open("spanish.txt")
stop_words_spanish = f.readlines()
f.close()
stop_words_spanish = [x.strip() for x in stop_words_spanish]

test_model_and_graph(NB_model, lang_class, stop_words_spanish, [lang_class['spanish'] for x in range(len(stop_words_spanish))], 
                  "Predictions for Spanish Stop Words")

In [ ]:
f = open("portuguese.txt")
stop_words_portuguese = f.readlines()
f.close()
stop_words_portuguese = [x.strip() for x in stop_words_portuguese]

test_model_and_graph(NB_model, lang_class, stop_words_portuguese, [lang_class['portuguese'] for x in range(len(stop_words_portuguese))], 
                  "Predictions for Portuguese Stop Words")

In [ ]:
f = open("english.txt")
stop_words_english = f.readlines()
f.close()
stop_words_english = [x.strip() for x in stop_words_english]

test_model_and_graph(NB_model, lang_class, stop_words_english, [lang_class['english'] for x in range(len(stop_words_english))], 
                  "Predictions for English Stop Words")

In [ ]:
sp_pt_overlap = set(stop_words_spanish).intersection(set(stop_words_portuguese))
sp_en_overlap = set(stop_words_spanish).intersection(set(stop_words_english))
en_pt_overlap = set(stop_words_english).intersection(set(stop_words_portuguese))
en_sp_pt_overlap = sp_pt_overlap.intersection(en_pt_overlap)

print("The actual overlap of Spanish and Portuguese words", len(sp_pt_overlap))
print("The actual overlap of Spanish and English words", len(sp_en_overlap))
print("The actual overlap of English and Portuguese words", len(en_pt_overlap))
print("The actual overlap of English, Spanish, and Portuguese words", len(en_sp_pt_overlap))

Most likley the reason that the model determines most of the time in favor of Portuguese is because the model does not "know" the words that it's being show and when it hasnt seen a word it is assigned the lowest probability 0 which is where poruguese was assigned its discreted value. 

We can investigate this later by changing the language classes from lang_class to lang_class_other and running the same test and seeing if the lanaguage more accuratley (or rather more often) predicts (in the case of lang_class_other) English. 

I did this intentionally because I noticed that a lot of times if GoogleTranslate isnt certain what language you're talking in, it defaults to English. While neither of these behaviours is ideal, the former reinforces that English is the default language. 

In [ ]:
train_NB_gold_other = [lang_class_other['portuguese'] for i in range(round(SCALE_LINES/2))]
train_NB_gold_other += [lang_class_other['spanish'] for i in range(round(SCALE_LINES/2))]
train_NB_gold_other += [lang_class_other['english'] for i in range(round(SCALE_LINES/2))]
print(len(train_NB_gold_other))

In [ ]:
NB_model_other = MultinomialNB()
NB_model_other.fit(train_NB_X, train_NB_gold_other)

In [ ]:
train_NB_pred_other = NB_model_other.predict(train_NB_X)
print("The accuracy is {}".format(accuracy_score(train_NB_gold_other, train_NB_pred_other)))
mat = confusion_matrix(train_NB_gold_other, train_NB_pred_other)
sns.heatmap(mat.T, square = True, annot=True, fmt = "d", xticklabels=[lang_class_other[0], lang_class_other[1], lang_class_other[2]],yticklabels=[lang_class_other[0], lang_class_other[1], lang_class_other[2]])
plt.title("Training Prediction Results for Language Classes where English is 0")
plt.xlabel("true labels")
plt.ylabel("predicted label")
plt.show()

Interestingly accuracy is now higher even thought all that was changed was the classes. From 96.9% to 97.2%. Its not that BIG of a difference, but a difference nonetheless

In [ ]:
NBmodel_smol_tests(NB_model_other, lang_class_other, ["Nicole", "Buckley", "Sierra", "Rodriguez"])
NBmodel_smol_tests(NB_model_other, lang_class_other, ["hello", "hola", "olá"])
NBmodel_smol_tests(NB_model_other, lang_class_other,  ["good morning", "buenos días", "bom dia"])

In [ ]:
test_model_and_graph(NB_model_other, lang_class_other, stop_words_english, [lang_class_other['english'] for x in range(len(stop_words_english))], 
                  "Predictions for English Stop Words for Other Lang Class")

Wow and thats what you call underresourced languages haha.... 

In [ ]:
test_model_and_graph(NB_model_other, lang_class_other, stop_words_spanish, [lang_class_other['spanish'] for x in range(len(stop_words_spanish))], 
                  "Predictions for Spanish Stop Words for Other Lang Class")

In [ ]:
test_model_and_graph(NB_model_other, lang_class_other, stop_words_portuguese, [lang_class_other['spanish'] for x in range(len(stop_words_portuguese))], 
                  "Predictions for Spanish Stop Words for Other Lang Class")

# RNN

Some references: 
https://www.tensorflow.org/text/tutorials/text_classification_rnn

https://www.tensorflow.org/text/tutorials/text_generation

https://www.tensorflow.org/guide/keras/rnn

https://www.google.com/search?client=safari&rls=en&q=how+to+make+a+tfds+dataset&ie=UTF-8&oe=UTF-8


In [ ]:
# todod data analysis 
#todo loiok at everage words per sentence comparnig the langauges

In [ ]:
# TODOO. move later
import tensorflow as tf
import unicodedata
import string
import keras

from keras.callbacks import Callback
! pip install scikit-plot
from scikitplot.metrics import plot_confusion_matrix, plot_roc
import os


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

EPOCHS = 5

## training the encoder and decoder

In [ ]:
RNN_corpus = " ".join(cleaned_portuguese[0:round(SCALE_LINES/2)]) + " ".join(cleaned_spanish[0:round(SCALE_LINES/2)]) + " ".join(cleaned_english[0:round(SCALE_LINES/2)])
RNN_vocab = list(set(RNN_corpus))
print(len(RNN_vocab))
# print(RNN_corpus[0:250])
print(RNN_vocab[0:100])
# print(RNN_vocab[100:200])
# print(RNN_vocab[200:300])
# print(RNN_vocab[300:342])

340
['Ō', 'ង', 'ò', 'ù', 'ú', '살', '만', '̀', 'υ', 'Ú', '세', '진', 'X', 'O', 'ج', '´', 'à', 'ן', 'ς', '你', 'ṑ', '屈', 'A', 'á', 'K', '\x94', 'ë', '해', '默', 'נ', '無', 'y', 'ខ', 'ñ', '로', 'λ', '視', '\x93', 'ុ', 'ŏ', 'v', 'ו', 'å', 'z', '។', 'Â', '後', 'ة', 'ញ', 'ت', '̂', 'æ', 'θ', '間', '鴻', 'ê', '어', '가', 't', 'Ñ', 'ه', 'p', '―', 'â', '預', 'ṡ', '足', 'ב', '憾', '撐', 'س', 'í', '̣', 'M', 'ì', '–', 'ε', '열', '天', '°', 'ك', 'º', 'ا', '군', 'ً', '말', 'ý', 'ą', 'ν', 'ث', 'R', 'ֶ', 'β', 'ق', 'ʼ', '■', 'þ', 'គ', 'w', 'ס']


In [ ]:
char_to_enc = tf.keras.layers.StringLookup(
    vocabulary=RNN_vocab)

In [ ]:
enc_to_char = tf.keras.layers.StringLookup(vocabulary=char_to_enc.get_vocabulary(), invert=True)

In [ ]:
def encs_to_text(encs):
  """
  encs: list of encodings to turn into full text
  """
  tensor =  tf.strings.reduce_join(enc_to_char(encs), axis=-1)
  sentences = tensor.numpy()
  sentences = [s.decode("UTF-8") for s in sentences]
  return sentences

## data set up

In [ ]:
train_RNN_sentences = cleaned_portuguese[0:round(SCALE_LINES/2)] + cleaned_spanish[0:round(SCALE_LINES/2)] + cleaned_english[0:round(SCALE_LINES/2)]
test_RNN_sentences = cleaned_english[-(round(SCALE_LINES/2)):] + cleaned_spanish[-(round(SCALE_LINES/2)):] + cleaned_portuguese[-(round(SCALE_LINES/2)):]

onehot_lang = {'portuguese': [1, 0, 0], 'spanish': [0, 1, 0], "english": [0, 0, 1]}

train_RNN_labelss = [onehot_lang['portuguese']] * round(SCALE_LINES/2) + [onehot_lang['spanish']] * round(SCALE_LINES/2)  + [onehot_lang['english']] * round(SCALE_LINES/2)
test_RNN_labelss = [onehot_lang['english']] * round(SCALE_LINES/2) + [onehot_lang['spanish']] * round(SCALE_LINES/2)  + [onehot_lang['portuguese']] * round(SCALE_LINES/2)

train_RNN_words_pt = [] 
train_RNN_words_sp = []
train_RNN_words_en = []

#notes that the words in test_sentences and test_words are NOT the same
test_RNN_words_pt = [] 
test_RNN_words_sp = [] 
test_RNN_words_en =  [] 

for i in range(round(SCALE_LINES/2)):
  train_RNN_words_pt += cleaned_portuguese[i].split(" ")
  train_RNN_words_sp += cleaned_spanish[i].split(" ")
  train_RNN_words_en += cleaned_english[i].split(" ")
  test_RNN_words_pt += cleaned_portuguese[i+round(SCALE_LINES/2)].split(" ")
  test_RNN_words_sp += cleaned_spanish[i+round(SCALE_LINES/2)].split(" ")
  test_RNN_words_en += cleaned_english[i+round(SCALE_LINES/2)].split(" ")

train_RNN_words = train_RNN_words_pt + train_RNN_words_sp + train_RNN_words_en
test_RNN_words = test_RNN_words_pt + test_RNN_words_sp + test_RNN_words_en

print("words in each language")
print("pt", len(test_RNN_words_pt))
print("sp", len(test_RNN_words_sp))
print("en", len(test_RNN_words_en))
print("total: " + str( len(test_RNN_words_pt) + len(test_RNN_words_sp) + len(test_RNN_words_en)))

train_RNN_labelsw = [onehot_lang['portuguese']] * len(train_RNN_words_pt) + [onehot_lang['spanish']] * len(train_RNN_words_sp)  + [onehot_lang['english']] * len(train_RNN_words_en)
test_RNN_labelsw = [onehot_lang['portuguese']] * len(test_RNN_words_pt) + [onehot_lang['spanish']] * len(test_RNN_words_sp)  + [onehot_lang['english']] * len(test_RNN_words_en)


#stplit all the sentences on each character and make into tensors
train_RNN_sentences_dec = tf.strings.unicode_split(
    train_RNN_sentences,
    input_encoding='UTF-8',
    name="training input"
)

train_RNN_words_dec = tf.strings.unicode_split(
    train_RNN_words,
    input_encoding='UTF-8',
    name="training input"
)

test_RNN_sentences_dec = tf.strings.unicode_split(
    test_RNN_sentences,
    input_encoding='UTF-8',
    name="training input"
)

test_RNN_words_dec = tf.strings.unicode_split(
    test_RNN_words,
    input_encoding='UTF-8',
    name="training input"
)

RNN_corpus_unicode = tf.strings.unicode_split(
    RNN_corpus,
    input_encoding='UTF-8',
    name="training corpus"
)

#encode all of the training/testing data 
train_RNN_sentences = char_to_enc(train_RNN_sentences_dec)
train_RNN_words = char_to_enc(train_RNN_words_dec)
test_RNN_sentences = char_to_enc(test_RNN_sentences_dec)
test_RNN_words = char_to_enc(test_RNN_words_dec)

words in each language
pt 485019
sp 431064
en 557274
total: 1473357


In [ ]:
# test the enc and decoder 

print("decoded sentences")
print(train_RNN_sentences_dec[0:3])
enc_test = char_to_enc(train_RNN_sentences_dec[0:3])

print("encoded sentences")
print(enc_test)
dec_test = enc_to_char(enc_test)

print("decoded sentences")
print(dec_test)

print("completley decoded sentences")
print(encs_to_text(enc_test))

decoded sentences
<tf.RaggedTensor [[b'E', b'u', b' ', b'n', b'\xc3\xa3', b'o', b' ', b't', b'i', b'n', b'h',
  b'a', b' ', b'e', b's', b't', b'e', b' ', b'r', b'o', b's', b't', b'o',
  b' ', b'd', b'e', b' ', b'h', b'o', b'j', b'e']                         ,
 [b'A', b's', b's', b'i', b'm', b' ', b'c', b'a', b'l', b'm', b'o', b' ',
  b'a', b's', b's', b'i', b'm', b' ', b't', b'r', b'i', b's', b't', b'e',
  b' ', b'a', b's', b's', b'i', b'm', b' ', b'm', b'a', b'g', b'r', b'o'],
 [b'N', b'e', b'm', b' ', b'e', b's', b't', b'e', b's', b' ', b'o', b'l',
  b'h', b'o', b's', b' ', b't', b'\xc3\xa3', b'o', b' ', b'v', b'a', b'z',
  b'i', b'o', b's']                                                       ]>
encoded sentences
<tf.RaggedTensor [[187, 203, 336, 185, 217, 133, 336, 59, 233, 185, 147, 177, 336, 312, 142,
  59, 312, 336, 240, 133, 142, 59, 133, 336, 226, 312, 336, 147, 133, 328,
  312]                                                                     ,
 [23, 142, 142, 233, 283, 33

In [ ]:
a = tf.data.Dataset.from_tensor_slices(train_RNN_labelss)
b = tf.data.Dataset.from_tensor_slices(train_RNN_sentences)
train_RNN_datasets =  tf.data.Dataset.zip((b, a))

for example, label in train_RNN_datasets.take(2):
  print(example)
  print(label)

a = tf.data.Dataset.from_tensor_slices(test_RNN_labelss)
b = tf.data.Dataset.from_tensor_slices(test_RNN_sentences)
test_RNN_datasets =  tf.data.Dataset.zip((b, a))

for example, label in test_RNN_datasets.take(2):
  print(example)
  print(label)

a = tf.data.Dataset.from_tensor_slices(train_RNN_labelsw)
b = tf.data.Dataset.from_tensor_slices(train_RNN_words)
train_RNN_datasetw =  tf.data.Dataset.zip((b, a))

a = tf.data.Dataset.from_tensor_slices(test_RNN_labelsw)
b = tf.data.Dataset.from_tensor_slices(test_RNN_words)
test_RNN_datasetw =  tf.data.Dataset.zip((b, a))




tf.Tensor(
[187 203 336 185 217 133 336  59 233 185 147 177 336 312 142  59 312 336
 240 133 142  59 133 336 226 312 336 147 133 328 312], shape=(31,), dtype=int64)
tf.Tensor([1 0 0], shape=(3,), dtype=int32)
tf.Tensor(
[ 23 142 142 233 283 336 163 177 220 283 133 336 177 142 142 233 283 336
  59 240 233 142  59 312 336 177 142 142 233 283 336 283 177 339 240 133], shape=(36,), dtype=int64)
tf.Tensor([1 0 0], shape=(3,), dtype=int32)
tf.Tensor(
[ 14 240 336  59 147 312 336 240 177 226 233 177 185  59 336 142 233 142
  59 312 240 142 336  59 147 312 336 182 220 312 233 177 226 312 142], shape=(35,), dtype=int64)
tf.Tensor([0 0 1], shape=(3,), dtype=int32)
tf.Tensor(
[ 23 220 220 336 133 203 240 336 240 133 177 226 142 336 339 133 336 185
 133  99 147 312 240 312], shape=(24,), dtype=int64)
tf.Tensor([0 0 1], shape=(3,), dtype=int32)


In [ ]:
#batch them
train_RNN_datasets = train_RNN_datasets.shuffle(4000)

for sample, label in test_RNN_datasets.take(1):
  print(sample)
  print(encs_to_text([sample]))
  print(label)

train_RNN_datasets = train_RNN_datasets.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_RNN_datasets = test_RNN_datasets.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_RNN_datasetw = train_RNN_datasets.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_RNN_datasetw = test_RNN_datasets.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

tf.Tensor(
[ 14 240 336  59 147 312 336 240 177 226 233 177 185  59 336 142 233 142
  59 312 240 142 336  59 147 312 336 182 220 312 233 177 226 312 142], shape=(35,), dtype=int64)
['Or the radiant sisters the Pleiades']
tf.Tensor([0 0 1], shape=(3,), dtype=int32)


## models set up

In [ ]:
#@title Simple RNN Model
RNN_model_simple = tf.keras.Sequential([
    # char_to_enc, # for some reaon when i try to add the encoder to the model, it doesnt like it for the sumamry
    tf.keras.layers.Embedding(
        input_dim=len(char_to_enc.get_vocabulary()),
        output_dim=BATCH_SIZE),
    tf.keras.layers.SimpleRNN(BATCH_SIZE),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(BATCH_SIZE, activation='relu'), 
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(3, activation='softmax')]) #have 3 outputs that are the probability matrix over the 3 langauge classes

# RNN_model_simple.summary()

In [ ]:
RNN_model_simple.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',
                       tf.keras.metrics.CategoricalAccuracy(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.Precision(),])

In [ ]:
RNN_model_simple.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          21824     
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 64)                8256      
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 3)                 195       
                                                                 
Total params: 34,435
Trainable params: 34,435
Non-trai

In [ ]:
#@title Bidirectional RNN Model
RNN_model_bi = tf.keras.Sequential([
    # char_to_enc, # for some reaon when i try to add the encoder to the model, it doesnt like it for the sumamry
    tf.keras.layers.Embedding(
        input_dim=len(char_to_enc.get_vocabulary()),
        output_dim=BATCH_SIZE),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(BATCH_SIZE, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')]) #have 3 outputs that are the probability matrix over the 3 langauge classes

RNN_model_simple.summary()

## train simple rnn

In [ ]:
RNN_model_simple.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',
                       tf.keras.metrics.CategoricalAccuracy(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.Precision(),])

In [ ]:
print(EPOCHS)

In [ ]:
RNN_history_simples = RNN_model_simple.fit(train_RNN_datasets, epochs=EPOCHS,
                    validation_data=test_RNN_datasets,
                    validation_steps=30,)

In [ ]:
plt.plot(RNN_history_simples.history['accuracy'])
plt.plot(RNN_history_simples.history['recall'])
plt.plot(RNN_history_simples.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Recall', 'Test'], loc='upper left')
plt.show()

### test some examples

In [ ]:
def output_to_lang(prob_matrix):
  """
  prob_matrix: a matrix with liklihood extimates for each language sape = (1, 3)

  returns string of the class representaaion of the highest probbiality index
  """
  p = list(prob_matrix)
  m = max(p)
  index = p.index(m)
  return "(" + str(m) + ") " + lang_class[index]

def RNNmodel_smol_tests(model, sentences):
  s = tf.strings.unicode_split(
    sentences,
    input_encoding='UTF-8')
  X = char_to_enc(s)
  pred = model.predict(X)
  [print(sentences[i], "is", output_to_lang(pred[i])) for i in range(len(sentences))]

RNNmodel_smol_tests(RNN_model_simple, ["Nicole", "Sierra", "Buckley", "Rodriguez"])
RNNmodel_smol_tests(RNN_model_simple, ["hello", "hola", "olá"])
RNNmodel_smol_tests(RNN_model_simple, ["good morning", "buenos días", "bom dia"])

RNNmodel_smol_tests(RNN_model_simple, ["Su novio está cumpliendo una larga condena.", 
                                       "Si me ayudas con el almuerzo, te haré un rico postre.", 
                                       "Si me ganara la lotería, me compraría una casa en Mallorca. ", 
                                       "Por favor, me avise quando devo descer", 
                                       "Eu gosto muito de viajar para lugares diferentes e experimentar comidas novas.", 
                                       "Ontem assisti a um filme muito emocionante que me fez chorar no final.", 
                                       "I really enjoy traveling to different places and trying new foods.", 
                                       "Today is a very hot day, so I'm going to the beach to enjoy the sun.",
                                       "Yesterday I watched a very emotional movie that made me cry at the end."])


## train bidirectional rnn

In [ ]:
RNN_model_bi.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',
                       tf.keras.metrics.CategoricalAccuracy(),
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.Precision(),])

In [ ]:
RNN_history_bis = RNN_model_bi.fit(train_RNN_datasets, epochs=1,
                    validation_data=test_RNN_datasets,
                    validation_steps=30,)

In [ ]:
plt.plot(RNN_history_bis.history['accuracy'])
plt.plot(RNN_history_bis.history['recall'])
plt.plot(RNN_history_bis.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Recall', 'Test'], loc='upper left')
plt.show()

### test some examples

In [ ]:
RNNmodel_smol_tests(RNN_model_bi, ["Nicole", "Sierra", "Buckley", "Rodriguez"])
RNNmodel_smol_tests(RNN_model_bi, ["hello", "hola", "olá"])
RNNmodel_smol_tests(RNN_model_bi, ["good morning", "buenos días", "bom dia"])

RNNmodel_smol_tests(RNN_model_bi, ["Su novio está cumpliendo una larga condena.", 
                                       "Si me ayudas con el almuerzo, te haré un rico postre.", 
                                       "Si me ganara la lotería, me compraría una casa en Mallorca. ", 
                                       "Por favor, me avise quando devo descer", 
                                       "Eu gosto muito de viajar para lugares diferentes e experimentar comidas novas.", 
                                       "Ontem assisti a um filme muito emocionante que me fez chorar no final.", 
                                       "I really enjoy traveling to different places and trying new foods.", 
                                       "Today is a very hot day, so I'm going to the beach to enjoy the sun.",
                                       "Yesterday I watched a very emotional movie that made me cry at the end."])
